In [9]:
%load_ext autoreload
%autoreload 2
import glob
import json

outs = {}


for filename in sorted([f for f in glob.glob("../output/*.json") if "test" not in f]): 
    model_name = os.path.basename(filename).split(".")[0]
    print(model_name)
    with open(filename) as f:
        outs[model_name] = json.load(f)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
beto-ft-1000
beto-ft-2000
beto


In [30]:
import pandas as pd
import numpy as np

resume = []

task_metrics = {
    "context_hate": ["eval_mean_f1", "eval_hate_f1"],
    "sentiment": ["eval_macro_f1", "eval_micro_f1"],
    "emotion": ["eval_macro_f1", "eval_micro_f1"],
}

for model_name, output in outs.items():
    line = {
        "model": model_name, 
    }

    for task, metrics in task_metrics.items():
        for metric in metrics:
            arr = np.array([evaluation[metric] for evaluation in output[task]])
            metric_name = metric.replace("eval_", "")
            line[task+" "+metric_name] = f"{arr.mean():.3f} +- {arr.std():.3f}"
    resume.append(line)

pd.DataFrame(resume)

,model,context_hate mean_f1,context_hate hate_f1,sentiment macro_f1,sentiment micro_f1,emotion macro_f1,emotion micro_f1
0,beto-ft-1000,0.586 +- 0.003,0.686 +- 0.002,0.672 +- 0.001,0.679 +- 0.004,0.534 +- 0.012,0.680 +- 0.020
1,beto-ft-2000,0.585 +- 0.005,0.687 +- 0.003,0.675 +- 0.005,0.679 +- 0.007,0.526 +- 0.010,0.696 +- 0.006
2,beto,0.591 +- 0.004,0.686 +- 0.006,0.652 +- 0.002,0.657 +- 0.003,0.525 +- 0.014,0.668 +- 0.016
